## This files combines the basic structures and logic of the implementation of user study, sequence is not correct, the final version will be in another file 

## You cannot drop any data anymore in this file

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
import statistics as stats
import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
import yaml
from tkinter import *
import tkinter
from geopy.geocoders import Nominatim
from geopy import distance
from geopy import Point
from scipy.sparse import csr_matrix, load_npz, save_npz

## Dataset files 

In [3]:
#reviewJson = "..\\data\\Export_CleanedReview.json"
#reviewJsonWithClosedRes = "..\\data\\Export_CleanedReviewWithClosedRes.json"
#reviewJsonToronto = "..\\data\\Export_TorontoData.json"


## Preprocess using Term Frequency - CounterVectorizer

## All Model

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, X, row_name = 'business_num_id', binary = True, shape = (121994,6000)):
    """
    get the item-keyphrase matrix
    """
    rows = []
    cols = []
    vals = []
    #For each review history
    for i in tqdm(range(X.shape[0])):
        #Get the array of frequencies for document/review i 
        arr = X[i].toarray() 
        nonzero_element = arr.nonzero()[1]  # Get nonzero element in each line, keyphrase that appears index 
        length_of_nonzero = len(nonzero_element) #number of important keyphrase that appears
        
        # df[row_name][i] is the item idex
        #Get a list row index that indicates the document/review
        rows.extend(np.array([df[row_name][i]]*length_of_nonzero)) ## Item index
        #print(rows)
        
        #Get a list of column index indicating the key phrase that appears in i document/review
        cols.extend(nonzero_element) ## Keyword Index
        if binary:
            #Create a bunch of 1s
            vals.extend(np.array([1]*length_of_nonzero))
        else:
            #If not binary 
            vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


#Get a UI matrix if it's not item_similarity based or else IU
def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        #Get prediction/similarity score for each user 1*num or user or num of items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        #Decending accoding to similarity score, select top k
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res


#Preidction score is UI or IU?
def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    #for each user
    for user_index in tqdm(range(matrix_Train.shape[0])):
        
        #take the prediction scores for user 1 * num res
        vector_u = prediction_score[user_index]
        
        #The restuarant the user rated
        vector_train = matrix_Train[user_index]
        
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)

#topK: the number of restuarants we are suggesting 
#if vector_train has number, then the user has visited
def sub_routine(vector_u, vector_train, topK=500):

    #index where non-zero
    train_index = vector_train.nonzero()[1]
    
    vector_u = vector_u
    
    #get topk + num rated res prediction score descending, top index 
    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    
    #sort top prediction score index in range topK+len(train_index) into vector_u`
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    
    #deleted the rated res from the topk+train_index prediction score vector for user u 
    #Delete the user rated res index from the topk+numRated index
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    #so we only include the top K prediction score here
    return vector_u[:topK]

## Methods for UI 

In [5]:
#Enter the user number that we are trying to recommend for
#Enter the # of businesses that we want to recommend for them 
#Pass in the UI_Prediction matrix for users 
#userIndex parameter not used 
def constructResDictionary(userIndex, busIndexRange, UI_prediction):
    #Construct the dictionary for the recommended restaurants to display 
    dictionaryToConstruct = {}
    
    #Loop through the number of businesses 
    for busIndex in range(busIndexRange):
        #Get the business information for the recommended business 
        businessSeries = df[df["business_num_id"] == UI_prediction[busIndex]].iloc[0]
        #Get the business name 
        busName = businessSeries['name']
        
        #get the list of strings to generate the address information 
        address_generator = (str(w) for w in yaml.safe_load(businessSeries.location)['display_address'])
        busLocation = ', '.join(address_generator)
        bus_Price = businessSeries.price
        busStars = businessSeries.business_stars
        busReviewCount = businessSeries.review_count_y 
        #category_generator = (str(s) for s in [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)])
        #busCategories = [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)]
        #busCategories = ', '.join(category_generator)
        busCategories = businessSeries.categories
        #Now add the restaurant to the dictionary
        dictionaryToConstruct[busName] = {'Address': busLocation,\
                                'Price': bus_Price,\
                                 'Star': busStars, \
                                 'Review Count': busReviewCount, \
                                 'Category': busCategories}
    return dictionaryToConstruct

In [6]:
def csr_vappend(a,b):
    """ Takes in 2 csr_matrices and appends the second one to the bottom of the first one. 
    Much faster than scipy.sparse.vstack but assumes the type to be csr and overwrites
    the first matrix instead of copying it. The data, indices, and indptr still get copied."""

    a.data = np.hstack((a.data,b.data))
    a.indices = np.hstack((a.indices,b.indices))
    a.indptr = np.hstack((a.indptr,(b.indptr + a.nnz)[1:]))
    a._shape = (a.shape[0]+b.shape[0],b.shape[1])
    return a

In [7]:
def get_three_popularity_matrix(df_original,rtrain):
    # get the list of popular items by ranking the number of reviews
    numUsers = rtrain.shape[0]
    numItems = rtrain.shape[1]
    
    dff_popular = df_original.copy()
    dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_num_of_reviews = dff_popular["business_num_id"].tolist()
    
    # get the list of popular items by ranking average rating score
    dff_popular_rating = df_original.copy()
    dff_popular_rating = dff_popular_rating.sort_values(by=["business_stars"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_avg_stars = dff_popular_rating["business_num_id"].tolist()
    
    lst_temp = []
    for item in tqdm(range(numItems)):
        numOfUsersRated = len(rtrain.toarray()[:, item].nonzero()[0])
        if numOfUsersRated <= 50:
            lst_temp.append(item)
    popular_list_avg_stars = [x for x in popular_list_avg_stars if x not in lst_temp]
    
    # get the popularity items by using the percentage liked method(number of liked items / total items)
    predictionMatrix = np.zeros((numUsers , numItems))

    # Define function for converting 1-5 rating to 0/1 (like / don't like)
    vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    rtrain_array = rtrain.toarray()
    # For every item calculate the number of people liked (4-5) divided by the number of people that rated
    itemPopularity = np.zeros((numItems))
    for item in range(numItems):
        numOfUsersRated = len(rtrain_array[:, item].nonzero()[0])
        numOfUsersLiked = len(vf(rtrain_array[:, item]).nonzero()[0])
#         if numOfUsersRated == 0:
        # set a threshold to filter out restaurants with very few reviews
        if numOfUsersRated <= 100:
            itemPopularity[item] = 0
        else:
            itemPopularity[item] = numOfUsersLiked/numOfUsersRated
    popular_list_liked_ratio = itemPopularity.argsort()
    
    return np.asarray(popular_list_num_of_reviews),np.asarray(popular_list_avg_stars),popular_list_liked_ratio

In [8]:
# either a list or an array
def geographical_dist(prediction_matrix,intersection,user_id, bus_indexRange):
    lst = []
    length = 0
    #if the prediction matrix is a list of items for an user 
    print(type(prediction_matrix))
    if isinstance(prediction_matrix, list):
        length = len(prediction_matrix)
    #loop through the prediction matrix for the user, if passed in a prediction matrix 
    else:
        length = prediction_matrix[user_id].shape[0]
        #length = prediction_matrix.shape[0]
    for j in range(length):
        if isinstance(prediction_matrix, list):
            coordinateDict = yaml.safe_load(df_location[df_location["business_num_id"] == prediction_matrix[j]].iloc[0].coordinates)
        else:    
            coordinateDict = yaml.safe_load(df_location[df_location["business_num_id"] == prediction_matrix[user_id][j]].iloc[0].coordinates)
        test_point = Point(coordinateDict['latitude'],coordinateDict['longitude'])
        
        #Get the distance with the test point
        result = distance.distance(intersection,test_point).kilometers
        if result<=0.6:
            #append the jth item if the condition matches
            if isinstance(prediction_matrix, list):
                lst.append(prediction_matrix[j])
            else:
                lst.append(prediction_matrix[user_id][j])
        lst = lst[0:bus_indexRange]
    return lst

In [9]:
class MultiListbox_Initialize(Frame):
    
    def __init__(self, master, lists,list1Res,list2Res,list3Res):
        Frame.__init__(self, master)
        self.lists = []
        #self.entries = []
        #self.fields = 'Like', 'Dislike'
        self.dropDownTextBox = 'Rank1','Score1', 'Rank2','Score2','Rank3','Score3'
        #Var for checkbox
        #self.var = IntVar()
        self.var = []
        self.listNames = []
        self.responseDict = {}
        #self.options = ["Jan", "Feb", "Mar"] #etc
        #No 0s or else you won't choose it as liked 
        self.scores = [5, 4, 3, 2, 1]
        #stores the list of restaurants for list 1-3
        self.list1Name = list1Res
        self.list2Name = list2Res
        self.list3Name = list3Res
        #To get response from dropdowns 
        self.rankVar = []
        self.scoreVar = []
        
        #list count 0,1,2
        listCount = 0 
        #Loop through the lists, l is the list label and widthW is the width 
        for l,widthW in lists:
            
            frame = Frame(self); 
            frame.pack(side=LEFT, expand=YES, fill=BOTH)
            
            Label(frame, text=l, borderwidth=1, relief=RAISED).pack(fill=X)
            
            #store the list names 
            self.listNames.append(l)
            
            lb = Listbox(frame, width=widthW, borderwidth=0, selectborderwidth=0,
                 relief=FLAT, exportselection=FALSE)
            lb.pack(expand=YES, fill=BOTH)
            #lb.pack(fill=X,expand=1)
            #lb.pack
            self.lists.append(lb)
            
            #lb.bind('<B1-Motion>', lambda e, s=self: s._select(e.y))
            #lb.bind('<Button-1>', lambda e, s=self: s._select(e.y))
            
            #lb.bind('<Leave>', lambda e: 'break')
            
            #lb.bind('<B2-Motion>', lambda e, s=self: s._b2motion(e.x, e.y))
            #lb.bind('<Button-2>', lambda e, s=self: s._button2(e.x, e.y))
             
            for fieldText in self.dropDownTextBox:

                if('Rank' in fieldText):
                    lab = Label(frame, text=fieldText, borderwidth=0, relief=RAISED)
                    lab.pack(side=TOP,fill=BOTH)
            
                    #Get the current list number 
                    if listCount+1 == 1:
                        currentOptions = self.list1Name
                    elif listCount+1 == 2:
                        currentOptions = self.list2Name
                    elif listCount+1 == 3:
                        currentOptions = self.list3Name
                        
                    variable = StringVar(master)
                    variable.set(currentOptions[0]) # default value

                    #row = Frame(self)
                    #lab = Label(frame, width=15, text=fieldText, anchor='center')
                    drowDwn = OptionMenu(frame, variable, *currentOptions)
                    
                    #Append the variable set from dropdown 
                    self.rankVar.append(variable)
                    
                    #lab.pack(side=LEFT,fill=X)
 
                    drowDwn.pack(side=TOP)


                elif('Score' in fieldText ):
                    variable = StringVar(master)
                    variable.set(self.scores[0]) # default value

                    #row = Frame(self)
                    lab = Label(frame, width=15, text=fieldText, anchor='center')
                    drowDwn = OptionMenu(frame, variable, *self.scores)
                    
                    #Append te varaible set from dropdown 
                    self.scoreVar.append(variable)

                    #lab.pack(side=RIGHT, fill=X)
                    drowDwn.pack(side=TOP)
            
            listCount += 1
            
        #Set the submit button  
        b1 = Button(master, text='Submit', command=self.fetch)
        b1.pack(side=BOTTOM, padx=5, pady=5)

        #pakcing the frame
        #frame = Frame(self); frame.pack(side=LEFT, fill=Y)
        
        #packing the label
        #Label(frame, borderwidth=1, relief=RAISED).pack(fill=X)

    def _select(self, y):
        row = self.lists[0].nearest(y)
        self.selection_clear(0, END)
        self.selection_set(row)
        return 'break'

    def _button2(self, x, y):
        for l in self.lists: l.scan_mark(x, y)
        return 'break'

    def _b2motion(self, x, y):
        for l in self.lists: l.scan_dragto(x, y)
        return 'break'

    def _scroll(self, *args):
        for l in self.lists:
            apply(l.yview, args)

    def curselection(self):
        return self.lists[0].curselection()

    def delete(self, first, last=None):
        for l in self.lists:
            l.delete(first, last)

#     def get(self, first, last=None):
#         result = []
#         for l in self.lists:
#             result.append(l.get(first,last))
#         if last: return apply(map, [None] + result)
#         return result

    def index(self, index):
        self.lists[0].index(index)

    def insert(self, index, *elements):
        #Loop through the elements 
        for element in elements:
            i = 0
            for l in self.lists:
                l.insert(index, element[i])
                i = i + 1

    def size(self):
        return self.lists[0].size()

    def see(self, index):
        for l in self.lists:
            l.see(index)

    def selection_anchor(self, index):
        for l in self.lists:
            l.selection_anchor(index)

    def selection_clear(self, first, last=None):
        for l in self.lists:
            l.selection_clear(first, last)

    def selection_includes(self, index):
        return self.lists[0].selection_includes(index)

    def selection_set(self, first, last=None):
        for l in self.lists:
            l.selection_set(first, last)
            
    def fetch(self):
        localDict = {}
        localRankList = list(map((lambda var: var.get()), self.rankVar))
        localScoreList = list(map((lambda var: var.get()), self.scoreVar))
        
        #lop through each selected restaurant and score, construct the dictionary 
        for count in range(len(localRankList)):
            localDict[localRankList[count]] = int(localScoreList[count])
            
        print('Your selected restaurants:', localRankList)
        print('Your scores given to the restaurants', localScoreList)
        print('Your final choice is :', localDict)
         
        self.responseDict = localDict.copy()
   

In [10]:
class MultiListbox_entries(Frame):
    
    def __init__(self, master, lists):
        Frame.__init__(self, master)
        self.lists = []
        self.entries = []
        self.fields = 'Like', 'Dislike'
        #Var for checkbox
        #self.var = IntVar()
        self.var = []
        self.listNames = []
        self.responseDict = {}
        
        #Loop through the lists, l is the list label and widthW is the width 
        for l,widthW in lists:
            
            frame = Frame(self); frame.pack(side=LEFT, expand=YES, fill=BOTH)
            
            #Label(frame, text=l, borderwidth=1, relief=RAISED).pack(fill=X)
            Label(frame, text='list', borderwidth=1, relief=RAISED).pack(fill=X)
            
            #store the list names 
            self.listNames.append(l)
            
            lb = Listbox(frame, width=widthW, borderwidth=0, selectborderwidth=0,
                 relief=FLAT, exportselection=FALSE)
            lb.pack(expand=YES, fill=BOTH)
            
            self.lists.append(lb)
            
            lb.bind('<B1-Motion>', lambda e, s=self: s._select(e.y))
            lb.bind('<Button-1>', lambda e, s=self: s._select(e.y))
            
            lb.bind('<Leave>', lambda e: 'break')
            
            lb.bind('<B2-Motion>', lambda e, s=self: s._b2motion(e.x, e.y))
            lb.bind('<Button-2>', lambda e, s=self: s._button2(e.x, e.y))
             
             #loop through the fields to 
            for field in self.fields:
                #row = Frame(self)
                lab = Label(frame, width=15, text=field, anchor='w')
                ent = Entry(frame)
                #row.pack(side=BOTTOM, fill=Y, padx=5, pady=5) #=BOTTOM
                lab.pack()
                #ent.pack(side=RIGHT, expand=YES, fill=Y)
                ent.pack(side=TOP, fill=X)
                self.entries.append((field, ent))
            
            #localVar = IntVar()
            
            #self.var.append(localVar)
            
            #Checkbox 
            #c = Checkbutton(frame, text="Liked", variable=localVar, command=self.cb(count))
            #c = Checkbutton(frame, text="Liked", variable=localVar)
            #c.pack()
            
            lab = Label(frame, width=15, text='List rank', anchor='w')
            ent = Entry(frame)
            #row.pack(side=BOTTOM, fill=Y, padx=5, pady=5) #=BOTTOM
            lab.pack()
            #ent.pack(side=RIGHT, expand=YES, fill=Y)
            ent.pack(side=TOP, fill=X)
            self.var.append((field, ent))
                
        #Set the submit button 
        #master.bind('<Button-1>', (lambda event, e=self.entries: fetch(e)))   
        #b1 = Button(master, text='Submit', command=(lambda e=self.entries: self.fetch(e)))
        b1 = Button(master, text='Submit', command=self.fetchDict)
        b1.pack(side=BOTTOM, padx=5, pady=5)
        #b2 = Button(root, text='Quit', command=root.quit)
        #b2.pack(side=LEFT, padx=5, pady=5)
            
        #pakcing the frame
        frame = Frame(self); frame.pack(side=LEFT, fill=Y)
        
        #packing the label
        Label(frame, borderwidth=1, relief=RAISED).pack(fill=X)
        #Setting a scrollbar 
#         sb = Scrollbar(frame, orient=VERTICAL, command=self._scroll)
#         sb.pack(expand=YES, fill=Y)
#         self.lists[0]['yscrollcommand']=sb.set

    def _select(self, y):
        row = self.lists[0].nearest(y)
        self.selection_clear(0, END)
        self.selection_set(row)
        return 'break'
    
    #For checkbox 
    def cb(self, index):
        print ("variable is", self.var[index].get())

    def _button2(self, x, y):
        for l in self.lists: l.scan_mark(x, y)
        return 'break'

    def _b2motion(self, x, y):
        for l in self.lists: l.scan_dragto(x, y)
        return 'break'

    def _scroll(self, *args):
        for l in self.lists:
            apply(l.yview, args)

    def curselection(self):
        return self.lists[0].curselection()

    def delete(self, first, last=None):
        for l in self.lists:
            l.delete(first, last)

#     def get(self, first, last=None):
#         result = []
#         for l in self.lists:
#             result.append(l.get(first,last))
#         if last: return apply(map, [None] + result)
#         return result

    def index(self, index):
        self.lists[0].index(index)

    def insert(self, index, *elements):
        #Loop through the elements 
        for element in elements:
            i = 0
            for l in self.lists:
                l.insert(index, element[i])
                i = i + 1

    def size(self):
        return self.lists[0].size()

    def see(self, index):
        for l in self.lists:
            l.see(index)

    def selection_anchor(self, index):
        for l in self.lists:
            l.selection_anchor(index)

    def selection_clear(self, first, last=None):
        for l in self.lists:
            l.selection_clear(first, last)

    def selection_includes(self, index):
        return self.lists[0].selection_includes(index)

    def selection_set(self, first, last=None):
        for l in self.lists:
            l.selection_set(first, last)
            
    def fetch(self,entries):
        count = 1
        
        for entry in entries:
            if count <= 2:
                listCount = 1
            elif count <= 4:
                listCount = 2
            elif count <= 6:
                listCount = 3
            elif count <= 8:
                listCount = 4
            elif count <= 10:
                listCount = 5             
            #print on odd number counts
            if count %2 != 0:
                print('list:', listCount)
            #index 0 is the field 
            #index 1 is the entry data 
            field = entry[0]
            text  = entry[1].get()
            print('%s: "%s"' % (field, text))
            
            count =count + 1
            
        print(list(map((lambda var: var.get()), self.var)))
   
    #This function returns a dictionary of the results
    def fetchDict(self):
        responseDict = {}
        count = 1
        
        #likeResult = list(map((lambda var: var.get()), self.var))
        
        for entry in self.entries:
            tempDict = {}
            
            field = entry[0]
            text  = entry[1].get()
            # like:"text content"
            tempDict[field] = text
            
            if count <= 2:
                listCount = 1
            elif count <= 4:
                listCount = 2
            elif count <= 6:
                listCount = 3
            elif count <= 8:
                listCount = 4
            elif count <= 10:
                listCount = 5      
                
            #update the dictionary with corresponding listname :  
            try:
                responseDict[self.listNames[listCount-1]].update(tempDict) 
            except:
                responseDict[self.listNames[listCount-1]] = tempDict 
            
            if count %2 != 0:
                #print on odd number counts
                print('list:', listCount)
                #Get the hit indicator and update the dictionary 
                ListRating = self.var[listCount-1][1].get()
                tempRateDict = {'listRating': ListRating}
                responseDict[self.listNames[listCount-1]].update(tempRateDict)
            
            #Print the current the field and entry text
            print('%s: "%s", list rating: %s' % (field, text,ListRating))
           
            count =count + 1
        
        #print the hit list
        #print(list(map((lambda var: var.get()), self.var)))
        
        self.responseDict = responseDict.copy()
        
        
    #Make the entry form 
    def makeform(root, fields):
        #For each field, like 
        for field in slef.fields:
            row = Frame(master)
            lab = Label(row, width=15, text=field, anchor='w')
            ent = Entry(row)
            row.pack(side=TOP, fill=X, padx=5, pady=5)
            lab.pack(side=LEFT)
            ent.pack(side=RIGHT, expand=YES, fill=X)
            self.entries.append((field, ent))
            
        self.entries =  entries

#### Get original dataframe out of the review datastet

In [141]:
df = pd.read_csv('..\\data\\userStudy\\dfWhole.csv') 
rtrain = load_npz('..\\data\\userStudy\\Rtrain.npz').tocsr()
rtrain_implicit = load_npz('..\\data\\userStudy\\Rtrain_implicit.npz').tocsr()

### Filtering out desserts and tea rooms specially for this user study

In [123]:
#df = df[df['categories'].str.contains('Barber','Desserts','Bakeries','Desserts, Bakeries')]
#restaurantFilter = df[df['name'].isin(selectedBusiness)].business_num_id.unique()

dontWantRestaurantString = "Barber|Desserts|Bakeries|Tea Rooms"

# We only take restaurant business with the above keywords
dontWant_restaurant = list(df[df["categories"].str.contains(dontWantRestaurantString,case=False)].business_num_id.unique())

dontWantRestaurantString2 = ['Coffee & Tea', 'Ice cream and Frozen Yogurt']
donWant2 = list(df[df["categories"].isin(dontWantRestaurantString2)].business_num_id.unique())

for busId in donWant2:
    dontWant_restaurant.append(busId)

In [124]:
#list(df[df["categories"].isin(dontWantRestaurantString2)].business_num_id.unique())

In [104]:
len(list(df[df["categories"].isin(dontWantRestaurantString2)].business_num_id.unique()))

105

In [125]:
type(dontWant_restaurant)

list

#### Get similarity matrices

In [13]:
#Store similarity matrices
# np.save("..\\data\\userStudy\\UU_explicit.npy",UU_similarity_explicit)
# np.save("..\\data\\userStudy\\UU_implicit.npy",UU_similarity_implicit)
# np.save("..\\data\\userStudy\\II_usingIU.npy",II_similarity_usingUI)
# np.save("..\\data\\userStudy\\II_usingIK.npy", II_similarity_usingIK)
# np.save("..\\data\\userStudy\\II_usingIC.npy",II_similarity_usingIC)

UU_similarity_implicit = np.load('..\\data\\userStudy\\UU_implicit.npy')
II_similarity_usingIK = np.load('..\\data\\userStudy\\II_usingIK.npy')
II_similarity_usingIC = np.load('..\\data\\userStudy\\II_usingIC.npy')



## Initial User Input Set Up

### 3 Popularity Metrics
#### get the popular items in three ways
1. avg stars
2. number of reviews
3. percentage liked

The small analysis and the map are in the Analyse_3_ways_of_popularities.ipynb

In [126]:
# number of reviews popularity list, redundent with the output of the next method
dff_popular = df.copy()
dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
#Get the list of restaurants accoridng to their popularity level
popular_list = dff_popular["business_num_id"].tolist()

In [127]:
# get number of users and number of items
numUsers = rtrain.shape[0]
numItems = rtrain.shape[1]

# get the 1d array of avg. stars, number of reviews and percentage liked ratio for the three popular metric
popular_list_num_of_reviews,popular_list_avg_stars,popular_list_liked_ratio = get_three_popularity_matrix(df,rtrain)

#Filter out restuarants not for meals
popular_list_num_of_reviews = np.asarray([x for x in popular_list_num_of_reviews if x not in dontWant_restaurant])
popular_list_avg_stars = np.asarray([x for x in popular_list_avg_stars if x not in dontWant_restaurant])
popular_list_liked_ratio = np.asarray([x for x in popular_list_liked_ratio if x not in dontWant_restaurant])

# transfer to a matrix(list * number of users)
matrix_popular_list_num_of_reviews = np.tile(popular_list_num_of_reviews,(numUsers+1,1))
#for propos, when taking in user data
matrix_popular_list_avg_stars = np.tile(popular_list_avg_stars,(numUsers+1,1))
#for recommendation
matrix_popular_list_liked_ratio = np.tile(popular_list_liked_ratio,(numUsers+1,1))


100%|██████████████████████████████████████████████████████████████████████████████| 3998/3998 [03:47<00:00, 16.88it/s]


### Recommend 3 lists for three locations by using "popular_list_liked_ratio" method

Note that the input of the geographical_dist method can only be list or n-d array

#### Initial setting

In [128]:
# get a copy of df
df_location = df.copy()

# three locations for user input
yonge_and_finch = Point("43.779824, -79.415665")
bloor_and_bathurst = Point("43.665194,-79.411208")
queen_and_spadina = Point("43.648772,-79.396259")

# three locations for user recommendation
bloor_and_yonge = Point("43.670409,-79.386814")
dundas_and_yonge = Point("43.6561,-79.3802")
spadina_and_dundas = Point("43.653004,-79.398082")

# list of percentage liked ratio (transfer the format in order to feed in to the geographical_dist method)
list_popular_list_avg_stars = popular_list_avg_stars.tolist()

# popularity list for input does not need to input the user_id
# put user_id as 0 here but it does not matter
# bus_indexRange = 15, gives a list of 15 restaurants per location
bus_indexRange = 15
yonge_and_finch_list = geographical_dist(list_popular_list_avg_stars,yonge_and_finch,0, bus_indexRange)
bloor_and_bathurst_list = geographical_dist(list_popular_list_avg_stars,bloor_and_bathurst,0, bus_indexRange)
queen_and_spadina_list = geographical_dist(list_popular_list_avg_stars,queen_and_spadina,0, bus_indexRange)

<class 'list'>
<class 'list'>
<class 'list'>


### List of restaurants for user demo 

In [129]:
# pd.set_option('display.max_columns', None)
# df_location_yonge_and_finch=df_location[df_location["business_num_id"].isin(yonge_and_finch_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name", "price", "categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
# df_location_yonge_and_finch.head(2)
# df_location_bloor_and_bathurst_list=df_location[df_location["business_num_id"].isin(bloor_and_bathurst_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","price","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
# df_location_bloor_and_bathurst_list.head(2)
# df_location_queen_and_spadina_list=df_location[df_location["business_num_id"].isin(queen_and_spadina_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","price","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
# df_location_queen_and_spadina_list.head(2)

### Construct the dictionary to be used for UI 

In [130]:
#Construct the dictionary to be used for UI 
#0 is hard coded, not used for now 
dict1_yongeFinch = constructResDictionary(0, bus_indexRange, yonge_and_finch_list)
dict2_bloorBathurst = constructResDictionary(0, bus_indexRange, bloor_and_bathurst_list)
dict3_queenSpadina = constructResDictionary(0, bus_indexRange, queen_and_spadina_list)


#These construct the restaurant name : business id dictionary for initial user setting
RestaurantBusId = {}

# yonge_and_finch_list
# bloor_and_bathurst_list
# queen_and_spadina_list
restYF = list(dict1_yongeFinch.keys())
restBB = list(dict2_bloorBathurst.keys())
restQS = list(dict3_queenSpadina.keys())

#loop through all items
for count in range(len(yonge_and_finch_list)):
    RestaurantBusId[restYF[count]] = yonge_and_finch_list[count]
    RestaurantBusId[restBB[count]] = bloor_and_bathurst_list[count]
    RestaurantBusId[restQS[count]] = queen_and_spadina_list[count]

#RestaurantBusId

#Validation 
#Get the business information for the recommended business 
# businessSeries = df[df["business_num_id"] == 1904].iloc[0]
# #Get the business name 
# businessSeries['name']

## Initial user setting

In [131]:
if __name__ == '__main__':
    #Initiate a frame or master? 
    tk = Tk()
    Label(tk, text='List of restaurants from 3 locations - choose 3 each').pack()
    #Label(tk, text='List of restaurants from 3 locations - choose 3 each').grid()
    
    #Creating the multi-listbox object, pass in a tuple of tuple (lists), this will be the list objects
    initialSetuUp = MultiListbox_Initialize(tk, (('Yonge & Finch Intersection', 20), ('Bloor & Bathurst Intersection', 20), \
                            ('Queen & Spadina Intersection', 20)),list(dict1_yongeFinch.keys()),list(dict2_bloorBathurst.keys()),\
                                 list(dict3_queenSpadina.keys()))

    #loop through the length of recommend list, # = 5 
    for index in range(len(dict1_yongeFinch)):
        #First restaurant information to display 
        #Following the restaurants' name 
        restList1 = list(dict1_yongeFinch.keys())[index]
        restList2 = list(dict2_bloorBathurst.keys())[index]
        restList3 = list(dict3_queenSpadina.keys())[index]
        
        initialSetuUp.insert(END, (' ', ' ', ' '))
        
        #Inserting the restaurant names
        initialSetuUp.insert(END, ('%d: %s' % (index + 1, restList1),'%d: %s' % (index + 1, restList2),
                         '%d: %s' % (index + 1, restList3)))
        
        #Looping through each attribute keys - resinfo
        for resinfo in dict1_yongeFinch.get(restList1).keys():
            restList1Info = resinfo + ':' + str(dict1_yongeFinch.get(restList1).get(resinfo,''))
            restList2Info = resinfo + ':' + str(dict2_bloorBathurst.get(restList2).get(resinfo,''))
            restList3Info = resinfo + ':' + str(dict3_queenSpadina.get(restList3).get(resinfo,''))
            
            initialSetuUp.insert(END, (restList1Info, restList2Info, restList3Info))
        
        initialSetuUp.insert(END, ('----------------', '----------------', '----------------'))
    
    initialSetuUp.pack(expand=YES,fill=BOTH)
    #initialSetuUp.grid(expand=YES,fill=BOTH)

    tk.mainloop()
    
    #Get user response
    UserInitialResponse = initialSetuUp.responseDict

Your selected restaurants: ['Sushi Bong', 'Artisan Noodle', 'Sang-Ji Fried Bao', "Little Piggy's", 'Mr.Tonkatsu', 'Sunrise House', 'Mira', 'Kiin', 'Khao San Road']
Your scores given to the restaurants ['5', '4', '3', '4', '5', '4', '5', '4', '4']
Your final choice is : {'Sushi Bong': 5, 'Artisan Noodle': 4, 'Sang-Ji Fried Bao': 3, "Little Piggy's": 4, 'Mr.Tonkatsu': 5, 'Sunrise House': 4, 'Mira': 5, 'Kiin': 4, 'Khao San Road': 4}


In [132]:
#Get the list of restaurant names the user has been to 
selectedBusiness  = list(UserInitialResponse.keys())

In [133]:
selectedBusiness

['Sushi Bong',
 'Artisan Noodle',
 'Sang-Ji Fried Bao',
 "Little Piggy's",
 'Mr.Tonkatsu',
 'Sunrise House',
 'Mira',
 'Kiin',
 'Khao San Road']

In [134]:
restaurantFilter = list(df[df['name'].isin(selectedBusiness)].business_num_id.unique())
#Add in the restaurants we don't want to recommend
for busId in dontWant_restaurant:
    restaurantFilter.append(busId)

In [135]:
restaurantFilter

[868,
 1643,
 1450,
 3371,
 2933,
 2399,
 680,
 2664,
 3482,
 1557,
 1363,
 54,
 926,
 2071,
 3661,
 869,
 2985,
 3914,
 2757,
 2143,
 3708,
 1839,
 2720,
 3327,
 3175,
 71,
 703,
 1396,
 3943,
 525,
 130,
 2891,
 2439,
 939,
 1964,
 719,
 408,
 1270,
 2842,
 193,
 1993,
 1917,
 3656,
 274,
 2331,
 1946,
 2949,
 947,
 649,
 2637,
 34,
 213,
 748,
 2326,
 442,
 2203,
 2817,
 3933,
 2781,
 2373,
 1915,
 1981,
 617,
 3517,
 3527,
 3809,
 3373,
 2687,
 16,
 1570,
 2721,
 3515,
 3764,
 662,
 2315,
 2352,
 702,
 3,
 516,
 1859,
 589,
 990,
 897,
 789,
 2545,
 1726,
 369,
 2458,
 3766,
 3507,
 1545,
 2402,
 3813,
 1437,
 2727,
 3684,
 3481,
 3064,
 2900,
 1323,
 1103,
 2205,
 3202,
 296,
 2573,
 3403,
 227,
 3106,
 1555,
 2413,
 255,
 3218,
 1503,
 2503,
 2988,
 3486,
 29,
 374,
 1496,
 2354,
 1518,
 2487,
 2266,
 791,
 161,
 3105,
 1315,
 11,
 2612,
 3098,
 302,
 2053,
 3946,
 1745,
 930,
 191,
 1476,
 671,
 3020,
 1484,
 2614,
 1728,
 3532,
 1287,
 878,
 3791,
 3291,
 1309,
 1932,
 3840,
 3

In [136]:
#Process user inputs
assert len(UserInitialResponse) ==9,"User response has duplicates!"

import statistics 
#Get row lis
rowList = [0] * len(UserInitialResponse)
#Get col list
colList = []
#Get data list
dataList = []

for resName, rating in UserInitialResponse.items():
    #Append the bus_num_id as column values
    #RestaurantBusId mapps restuarnt names to business ids
    colList.append(RestaurantBusId[resName])
    dataList.append(rating)

userAvg = statistics.mean(dataList) 
dataWuserAvgList = np.array(dataList) - np.array([userAvg] * len(UserInitialResponse))+ 0.001
rows = np.array(rowList)
cols = np.array(colList)
data = np.array(dataList)
dataWuser = np.array(dataWuserAvgList) 
print(rows, cols,data)

#Get explicit data
userSetUpMatrix = csr_matrix((data, (rows, cols)), shape=(1, rtrain.shape[1]))
#Get with user rating
userSetUpMatrix_WuserAvg = csr_matrix((dataWuser, (rows, cols)), shape=(1, rtrain.shape[1]))

#Generate Implicit user rating vector
implicitUserSetUpMtx = csr_matrix((data, (rows, cols)), shape=(1, rtrain.shape[1]))
implicitUserSetUpMtx[(userSetUpMatrix > 3).nonzero()] = 1

[0 0 0 0 0 0 0 0 0] [1643 3371 2664  680 1450 3482 1363 1557 2399] [5 4 3 4 5 4 5 4 4]


## !Stacking new user data with original data, ONLY RUN ONCE!

In [142]:
#This would overwrite rtrain,rtrain_implicit, and rtrain_WuserAvg
csr_vappend(rtrain,userSetUpMatrix)
csr_vappend(rtrain_implicit, implicitUserSetUpMtx)
#csr_vappend(rtrain_userAvg, userSetUpMatrix_WuserAvg)

#User_index = #This should be the user index rtrain.shape[0]

<6101x3998 sparse matrix of type '<class 'numpy.float64'>'
	with 180041 stored elements in Compressed Sparse Row format>

### Compute similarity score for new user

#### Update rtrain similarity 

In [143]:
#6100 * 6100
#should import 
initial_rtrain_similarity = np.load('..\\data\\userStudy\\UU_explicit.npy')

#Compute the similarity vector for the new user 1*6101
newSimilarityVector_explicit = cosine_similarity(X=userSetUpMatrix, Y=rtrain, dense_output=True)

#new user similarity row
# 1 * 6100
newSimilarity_row = newSimilarityVector_explicit[0][:-1]
newSimilarity_row = np.expand_dims(newSimilarity_row, axis=0)

#new user similarity column
# 6101 * 1
newSimilarity_col = newSimilarityVector_explicit.T

In [144]:
newSimilarity_row.shape

(1, 6100)

In [145]:
#stack the bottom row
new_rtrain_similarity = np.vstack((initial_rtrain_similarity, newSimilarity_row))

#stack the last column
#new trains similarity: 6101 * 6100, new similarity_col: 6101 * 1
new_rtrain_similarity = np.hstack((new_rtrain_similarity, newSimilarity_col))

#validation
#(new_rtrain_similarity[-1,:] == newSimilarity_row).sum()

#### Update rtrain_implicit similarity 

In [146]:
#Initial rtrain implicit similarity: 6100 * 6100
#should import 
initial_rtrainImplicit_similarity = np.load('..\\data\\userStudy\\UU_implicit.npy')

#Compute the similarity vector for the new user 1*6101
newSimilarityVector_implicit = cosine_similarity(X=implicitUserSetUpMtx, Y=rtrain_implicit, dense_output=True)

In [147]:
#new user similarity row
# 1 * 6100
newSimilarity_implicit_row = newSimilarityVector_implicit[0][:-1]
newSimilarity_implicit_row = np.expand_dims(newSimilarity_implicit_row, axis=0)

#new user similarity column
# 6101 * 1
newSimilarity_implicit_col = newSimilarityVector_implicit.T

In [148]:
#stack the bottom row
new_rtrainImplicit_similarity = np.vstack((initial_rtrainImplicit_similarity, newSimilarity_implicit_row))

#stack the last column
#new trains similarity: 6101 * 6100, new similarity_col: 6101 * 1
new_rtrainImplicit_similarity = np.hstack((new_rtrainImplicit_similarity, newSimilarity_implicit_col))

## User-rating KNN

### 1. Implicit User-rating

In [149]:
#UU similarity, using cosine similarity
#similarity_2 = train(rtrain_implicit)
UI_predictionScore_Implicit = predict(rtrain_implicit, 100, new_rtrainImplicit_similarity, item_similarity_en= False)
UI_predict_Implicit = prediction(UI_predictionScore_Implicit, 50, rtrain_implicit)

100%|████████████████████████████████████████████████████████████████████████████| 6101/6101 [00:01<00:00, 4109.47it/s]


In [150]:
#Filter out visited restaurants
tr = UI_predict_Implicit[rtrain.shape[0]-1]
tr = [x for x in tr if x not in restaurantFilter]
lenDiff = len(UI_predict_Implicit[rtrain.shape[0]-1]) - len(tr)
tr = tr + [0] * lenDiff
UI_predict_Implicit[rtrain.shape[0]-1] = np.array(tr)

### 2. Implicit similarity, Explicit user-rating combination


In [151]:
#similarity_3 = train(rtrain_implicit)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
UI_predictionScore_IECombined = predict(rtrain, 100, new_rtrainImplicit_similarity, item_similarity_en= False)
UI_predict_IECombined = prediction(UI_predictionScore_IECombined, 50, rtrain)
# user_item_res1 = evaluate(UI_predict_IECombined, rvalid)
# user_item_res1

100%|████████████████████████████████████████████████████████████████████████████| 6101/6101 [00:01<00:00, 4062.06it/s]


In [152]:
#Filter out visited restaurants
tr = UI_predict_IECombined[rtrain.shape[0]-1]
tr = [x for x in tr if x not in restaurantFilter]
lenDiff = len(UI_predict_IECombined[rtrain.shape[0]-1]) - len(tr)
tr = tr + [0] * lenDiff
UI_predict_IECombined[rtrain.shape[0]-1] = np.array(tr)

### 3. Item_based TF-IDF

In [153]:
II_similarity_IK = np.load('..\\data\\userStudy\\II_usingIK.npy')
item_based_prediction_score4 = predict(rtrain, 10, II_similarity_IK, item_similarity_en= True)
#for each restuarant top50 users 
Item_predict_tfidf = prediction(item_based_prediction_score4, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6101/6101 [00:01<00:00, 3602.32it/s]


In [154]:
#Filter out visited restaurants
tr = Item_predict_tfidf[rtrain.shape[0]-1]
tr = [x for x in tr if x not in restaurantFilter]
lenDiff = len(Item_predict_tfidf[rtrain.shape[0]-1]) - len(tr)
tr = tr + [0] * lenDiff
Item_predict_tfidf[rtrain.shape[0]-1] = np.array(tr)

### 4. Item_based IC

In [155]:
item_based_prediction_IC = predict(rtrain, 10, II_similarity_usingIC, item_similarity_en= True)
#for each restuarant top50 users 
Item_predict_IC = prediction(item_based_prediction_IC, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6101/6101 [00:01<00:00, 3782.51it/s]


In [156]:
#Filter out visited restaurants
tr = Item_predict_IC[rtrain.shape[0]-1]
tr = [x for x in tr if x not in restaurantFilter]
lenDiff = len(Item_predict_IC[rtrain.shape[0]-1]) - len(tr)
tr = tr + [0] * lenDiff
Item_predict_IC[rtrain.shape[0]-1] = np.array(tr)

## Recommendation Section: Produce the list of restaurants close to the set location

#### ***Remember to manually change this userIndex!!!***

In [157]:
import random
# three locations for user recommendation
intersection = [bloor_and_yonge,dundas_and_yonge,spadina_and_dundas]
# get 3 items for each location
businessIndexRange = 3
# 5 metrics and store them in a list
metric = [UI_predict_Implicit, matrix_popular_list_liked_ratio, UI_predict_IECombined, Item_predict_IC, Item_predict_tfidf]

# manually enter this number!!!!!!
userIndex = rtrain.shape[0]

#Need this number to perform the user test, so don't repeat, don't mess up
userTestNumber = random.randint(1,1001)

In [158]:
print('********************** userIndex:',userIndex, 'userTestNumber on file:', userTestNumber,'**********************')

********************** userIndex: 6101 userTestNumber on file: 328 **********************


#### Get the recommendations for each metric using loops, store in the res

In [159]:
# get 3 top recommendations for each locations
# store three locations in one list for each metric

#loop through the 5 recommendation algorithms, initialize the res list 
res = [[] for i in range(len(metric))]
#Loop through the intersections 
for i in range(len(intersection)):
    #loop through each metric 
    for j in range(len(metric)):
        temp = geographical_dist(metric[j],intersection[i],userIndex-1,businessIndexRange)
        res[j] += temp

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [160]:
# top 10 recommendations for each metric
# the sequence is -> [UI_predict_Implicit, popular_list, UI_predict_IECombined, IU_predict_Explicit, Item_predict_tfidf]
# get three recommendations for each location, so the len for each row is 3*3
res

[[1252, 2159, 2938, 2857, 1490],
 [2604, 2564, 2486, 2652, 2643, 2535, 2602, 2659, 2669],
 [1252, 2159, 2938, 2857, 1490],
 [2740, 2652, 1150, 1459],
 [3553, 3497, 2264, 363, 2882, 3595]]

#### Look back the the origal prediction matrix and rearrange the recommendations list, choose the top 3

In [161]:
res_final = []
#loop through the elements in the res list (5 recommend lists)
for element in range(len(res)):
    dic = {}
    #loop through each element in the recomemndation list (order:element)
    for i in range(len(res[element])):
        #if the recommendation is a list 
        if isinstance(metric[element], list):
            dic[metric[element].index(res[element][i])] = res[element][i]
        #if the recommendation is a matrix 
        else:
            dic[metric[element][userIndex-1].tolist().index(res[element][i])] = res[element][i]
    temp = []
    for j in sorted(dic.keys()):
        temp.append(int(dic[j]))
    res_final.append(temp[0:3])

In [162]:
# the final recommendations for each metric
res_final

[[2938, 2857, 2159],
 [2602, 2604, 2564],
 [2938, 2857, 2159],
 [1150, 2740, 1459],
 [3553, 363, 3497]]

## Match restaurant information according to business_num_id

#### Current sequence: UI_predict_Implicit, popular_list, UI_predict_IECombined, IU_predict_Explicit, Item_predict_tfidf

In [163]:
metric = [UI_predict_Implicit, matrix_popular_list_avg_stars, UI_predict_IECombined, Item_predict_IC, Item_predict_tfidf]
#Trying to recommend for user 0 for now 
dict1_ImplicitRecommend = {}
dict2_PopularityRecommend = {}
dict3_EICombineRecommend = {}
dict4_IIbasedICRecommend = {}
dict5_IIbasedTFIDFRecommend = {}
#User predict [user index] [item index]
#Add a for loop for the top recommended items

dict1_ImplicitRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[0])
dict2_PopularityRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[1])
dict3_EICombineRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[2])
dict4_IIbasedICRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[3])
dict5_IIbasedTFIDFRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[4])

### Sequence for now: UI_predict_Implicit, popular_list, UI_predict_IECombined, IU_predict_Explicit, Item_predict_tfidf 

In [164]:
if __name__ == '__main__':
    #Initiate a frame or master? 
    tk = Tk()
    Label(tk, text='List of recommended restaurants').pack()
    
    #Creating the multi-listbox object, pass in a tuple of tuple (lists), this will be the list objects
    mlb = MultiListbox_entries(tk, (('Implicit User-Rating', 20), ('Popularity List', 20), \
                            ('Explicit Implicit Combined', 20), ('Explicit Item-Rating', 20), ('TF-IDF Score of Item', 20)))

    #loop through the length of recommend list, # = 5 
    for index in range(len(dict1_ImplicitRecommend)):
        #First restaurant information to display 
        #Following the restaurants' name 
        restList1 = list(dict1_ImplicitRecommend.keys())[index]
        restList2 = list(dict2_PopularityRecommend.keys())[index]
        restList3 = list(dict3_EICombineRecommend.keys())[index]
        restList4 = list(dict4_IIbasedICRecommend.keys())[index]
        restList5 = list(dict5_IIbasedTFIDFRecommend.keys())[index]
        
        mlb.insert(END, (' ', ' ', ' ', ' ', ' '))
        
        #Inserting the restaurant names
        mlb.insert(END, ('%d: %s' % (index + 1, restList1),'%d: %s' % (index + 1, restList2),
                         '%d: %s' % (index + 1, restList3),'%d: %s' % (index + 1, restList4),
                         '%d: %s' % (index + 1, restList5)))
        
        #Looping through each attribute keys - resinfo
        for resinfo in dict1_ImplicitRecommend.get(restList1).keys():
            restList1Info = resinfo + ':' + str(dict1_ImplicitRecommend.get(restList1).get(resinfo,''))
            restList2Info = resinfo + ':' + str(dict2_PopularityRecommend.get(restList2).get(resinfo,''))
            restList3Info = resinfo + ':' + str(dict3_EICombineRecommend.get(restList3).get(resinfo,''))
            restList4Info = resinfo + ':' + str(dict4_IIbasedICRecommend.get(restList4).get(resinfo,''))
            restList5Info = resinfo + ':' + str(dict5_IIbasedTFIDFRecommend.get(restList5).get(resinfo,''))
            
            mlb.insert(END, (restList1Info, restList2Info, restList3Info, restList4Info, restList5Info))
        
        mlb.insert(END, ('----------------', '----------------', '----------------', '----------------', '----------------'))
    
    mlb.pack(expand=YES,fill=BOTH)

    tk.mainloop()
    
    #Get user response
    response = mlb.responseDict

list: 1
Like: "asdfa", list rating: 3
Dislike: "sdf", list rating: 3
list: 2
Like: "asdf", list rating: 4
Dislike: "asdf", list rating: 4
list: 3
Like: "asdf", list rating: 1
Dislike: "asdf", list rating: 1
list: 4
Like: "asdf", list rating: 2
Dislike: "asdf", list rating: 2
list: 5
Like: "asdf", list rating: 5
Dislike: "asdf", list rating: 5


In [165]:
#Get user response
response = mlb.responseDict

In [166]:
response['userinitialResponse'] = UserInitialResponse

In [167]:
response

{'Implicit User-Rating': {'Like': 'asdfa',
  'listRating': '3',
  'Dislike': 'sdf'},
 'Popularity List': {'Like': 'asdf', 'listRating': '4', 'Dislike': 'asdf'},
 'Explicit Implicit Combined': {'Like': 'asdf',
  'listRating': '1',
  'Dislike': 'asdf'},
 'Explicit Item-Rating': {'Like': 'asdf',
  'listRating': '2',
  'Dislike': 'asdf'},
 'TF-IDF Score of Item': {'Like': 'asdf',
  'listRating': '5',
  'Dislike': 'asdf'},
 'userinitialResponse': {'Sushi Bong': 5,
  'Artisan Noodle': 4,
  'Sang-Ji Fried Bao': 3,
  "Little Piggy's": 4,
  'Mr.Tonkatsu': 5,
  'Sunrise House': 4,
  'Mira': 5,
  'Kiin': 4,
  'Khao San Road': 4}}

In [168]:
recommendationDict = {}
recommendationDict['ImplicitRecommend'] = dict1_ImplicitRecommend
recommendationDict['PopularityRecommend'] = dict2_PopularityRecommend
recommendationDict['EICombineRecommend'] = dict3_EICombineRecommend
recommendationDict['IIbasedICRecommend'] = dict4_IIbasedICRecommend
recommendationDict['IIbasedTFIDFRecommend'] = dict5_IIbasedTFIDFRecommend

# Saving the response


In [169]:
#store the user response into local file
csv_fileName = "UserTestResult{:d}.json".format(userTestNumber)
with open('userStudyResults//'+csv_fileName, 'w') as fp:
    json.dump(response, fp)

In [170]:
import csv
#store the user recommendation into local file
csv_fileName = "UserTestRecommendation{:d}.csv".format(userTestNumber)
w = csv.writer(open("output.csv", "w"))
for key, val in recommendationDict.items():
    w.writerow([key, val])
    

### I get the user response information here, so please make sure the sequence for list names are correct when initializing our mlb object!

In [171]:
#To load the result file with json 
# with open(csv_fileName, 'r') as fp:
#     test = json.load(fp)

In [172]:
#Get the 0th user review data 
rows, cols = rtrain[rtrain.shape[0]-1].nonzero()
cols

array([ 680, 1363, 1450, 1557, 1643, 2399, 2664, 3371, 3482])

In [173]:
#Get the corresponding business information using the business num id 
pd.set_option('display.max_colwidth', -1)
df[df["business_num_id"].isin(cols)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})

,business_num_id,name,categories,business_stars,review_count
42716,1643,Sushi Bong,"Sushi Bars, Japanese",4.0,272
56469,1450,Mr.Tonkatsu,Japanese,4.0,174
65585,3371,Artisan Noodle,Chinese,3.5,106
72222,2399,Khao San Road,Thai,4.0,1428
129109,680,Little Piggy's,"Korean, Barbeque",4.0,133
134039,2664,Sang-Ji Fried Bao,"Noodles, Dim Sum, Soup",4.0,93
161798,3482,Sunrise House,Korean,4.0,183
161965,1557,Kiin,Thai,4.0,150
186801,1363,Mira,Peruvian,4.5,103
